In [1]:
import torch
print(f"Version of torch - {torch.__version__}")
print(f"Version of cuda - {torch.version.cuda}")
print(f'Is CUDA available - {torch.cuda.is_available()}')

C:\Users\Daniil\anaconda3\envs\newpy36\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version of torch - 1.10.2+cpu
Version of cuda - None
Is CUDA available - False


In [2]:
import timm
print(f'Version of timm - {timm.__version__}')

ImportError: cannot import name 'container_abcs'

In [7]:
# -*- coding: utf-8 -*-
from util.My_tool1 import *
import time
import torch
import numpy as np
import matplotlib.pyplot as plt

if __name__ == '__main__':

    # Загрузка модели
    model = torch.load(r'C:\Users\Daniil\PycharmProjects\SwinTransformer\trained_model\model.pth', map_location=torch.device('cpu'))

    model.eval()  # Evaluation mode

    # Загрузка тестовых данных
    x = np.load('test_data/clear.npy')
    x = x.astype(np.float64)

    y = np.load('test_data/noise_and_miss.npy')
    y_ = torch.from_numpy(y).view(1, -1, y.shape[0], y.shape[1])

    # Убедитесь, что данные в формате float32
    y_ = y_.type(torch.float32)

    # Измерение времени
    start_time = time.time()

    # Инференс модели
    x_ = model(y_)  # Inference
    x_ = x_.view(y.shape[0], y.shape[1])
    x_ = x_.detach().numpy().astype(np.float64)

    elapsed_time = time.time() - start_time

    # Расчет метрик
    pre_snr = snr_(y, x)
    print("before：snr " + str(pre_snr))
    snr = snr_(x_, x)
    print("After：snr " + str(snr))

    # Исправленный вызов ssim_
    pre_ssim = ssim_(y, x, data_range=1.0)  # Указываем data_range
    print("before：ssim " + str(pre_ssim))
    ssim = ssim_(x_, x, data_range=1.0)  # Указываем data_range
    print("After：ssim " + str(ssim))

    # Вычисление MSE между исходными и обработанными данными
    mse = np.mean((x - x_) ** 2)
    print("MSE between original and denoised data: ", mse)

    # Визуализация данных
    plt.imshow(x, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Исходные данные")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()

    plt.imshow(y, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Прореженные данные")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()

    plt.imshow(x_, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Результат работы нейросети")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()

    plt.imshow(x - x_, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Разница")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()


ImportError: cannot import name 'container_abcs'

In [5]:
# -*- coding: utf-8 -*-
from util.My_tool1 import *
import time
import torch
import numpy as np
import matplotlib.pyplot as plt


if __name__ == '__main__':

    # model = torch.load(r'trained_model\model.pth', map_location=torch.device('cpu'))
    model = torch.load(r'trained_model\model.pth')

    model.eval()  # evaluation mode
    if torch.cuda.is_available():
        model = model.cuda()

    x = np.load('test_data/clear.npy')
    x = x.astype(np.float64)

    y = np.load('test_data/noise_and_miss.npy')
    y_ = torch.from_numpy(y).view(1, -1, y.shape[0], y.shape[1])

    torch.cuda.synchronize()
    start_time = time.time()
    y_ = y_.type(torch.float32)
    y_ = y_.cuda()

    x_ = model(y_)  # inferences
    x_ = x_.view(y.shape[0], y.shape[1])
    x_ = x_.cpu()
    x_ = x_.detach().numpy().astype(np.float64)
    torch.cuda.synchronize()
    elapsed_time = time.time() - start_time

    pre_snr = snr_(y, x)
    print("before：snr" + str(pre_snr))
    snr = snr_(x_, x)
    print("After：snr" + str(snr))

    # Исправленный вызов ssim_
    pre_ssim = ssim_(y, x, data_range=1.0)  # Указываем data_range
    print("before：ssim" + str(pre_ssim))
    ssim = ssim_(x_, x, data_range=1.0)  # Указываем data_range
    print("After：ssim" + str(ssim))

    # Calculate MSE between original and denoised data
    mse = np.mean((x - x_) ** 2)
    print("MSE between original and denoised data: ", mse)

    # Вывод графиков в сером цвете
    plt.imshow(x, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Исходные данные")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()

    plt.imshow(y, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Прореженные данные")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()

    plt.imshow(x_, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Результат работы нейросети")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()

    plt.imshow(x-x_, cmap='gray', aspect='auto', vmin=-1, vmax=1)
    plt.title("Разница")
    plt.xlabel('Расстояние от источника, м')
    plt.ylabel('Время свободного пробега, мс')
    plt.show()


ImportError: cannot import name 'container_abcs'

In [6]:
# -*- coding: utf-8 -*-
from util.My_tool1 import *
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
import os

# Установить переменную среды для обхода конфликта OpenMP
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

if __name__ == '__main__':
    model = torch.load('trained_model/model.pth')
    model.eval()  # evaluation mode
    model = model.cpu()  # Использовать CPU

    x = np.load('test_data/clear.npy')
    x = x.astype(np.float64)

    y = np.load('test_data/noise_and_miss.npy')
    y_ = torch.from_numpy(y).view(1, -1, y.shape[0], y.shape[1])
    y_ = y_.type(torch.float32)
    y_ = y_.cpu()  # CPU

    start_time = time.time()
    x_ = model(y_)  # inference
    x_ = x_.view(y.shape[0], y.shape[1])
    x_ = x_.detach().numpy().astype(np.float64)
    elapsed_time = time.time() - start_time

    pre_snr = snr_(y, x)
    print("before：snr " + str(pre_snr))
    snr = snr_(x_, x)
    print("After：snr " + str(snr))

    pre_ssim = ssim_(y, x)
    print("before：ssim " + str(pre_ssim))
    ssim = ssim_(x_, x)
    print("After：ssim " + str(ssim))

    imgplot1 = plt.imshow(x, cmap=plt.cm.seismic, aspect='auto', vmin=-1, vmax=1)
    plt.show()

    imgplot1 = plt.imshow(y, cmap=plt.cm.seismic, aspect='auto', vmin=-1, vmax=1)
    plt.show()

    imgplot1 = plt.imshow(x_, cmap=plt.cm.seismic, aspect='auto', vmin=-1, vmax=1)
    plt.xlabel('Trace')
    plt.ylabel('Samples')
    plt.show()


ImportError: cannot import name 'container_abcs'